In [ ]:
import time

In [ ]:
start_time = time.time()

lista_numeros = []

for i in range(0, 1000001):
    
    lista_numeros.append(i)

final_time = time.time() - start_time

print(f"El algoritmo demoró: {final_time}")

In [ ]:
start_time = time.time()

lista_numeros2 = [i**2 for i in range(0, 1000001)]

final_time = time.time() - start_time

print(f"El algoritmo demoró: {final_time}")

In [ ]:
import sqlite3
import random
import time
import os

In [ ]:
conn = sqlite3.connect(os.path.join(os.getcwd(), "clientes.sqlite3"))  # Base de datos en memoria
cur = conn.cursor()

In [ ]:
cur.execute('''CREATE TABLE Clientes (
                id INTEGER PRIMARY KEY,
                nombre TEXT,
                ciudad TEXT)''')


In [ ]:
def generar_nombre():
    nombres = ['Juan', 'Maria', 'Matias', 'Ana', 'Carlos', 'Lucia', 'Diego', 'Laura']
    return random.choice(nombres) + ' ' + random.choice(['Arroyo', 'Macchi', 'Lopez', 'Martinez'])


In [ ]:
ciudades = ['Buenos Aires', 'Córdoba', 'Rosario', 'Mendoza', 'Salta', 'La Plata', 'Mar del Plata', 'Jujuy']

for i in range(1000):
    nombre = generar_nombre()
    ciudad = random.choice(ciudades)
    cur.execute('INSERT INTO Clientes (nombre, ciudad) VALUES (?, ?)', (nombre, ciudad))


In [ ]:
conn.commit()

In [ ]:
ciudad_busqueda = 'Jujuy'
consulta = f"SELECT * FROM Clientes WHERE ciudad = '{ciudad_busqueda}'"

start_time = time.time()
cur.execute(consulta)
resultados_sin_indice = cur.fetchall()
tiempo_sin_indice = time.time() - start_time

print(f'Tiempo de ejecución sin índice: {tiempo_sin_indice:.6f} segundos')
print(f'Número de resultados sin índice: {len(resultados_sin_indice)}')


In [ ]:
# Paso 4: Crear un índice en la columna ciudad
cur.execute('CREATE INDEX idx_ciudad ON Clientes (ciudad)')
conn.commit()

In [ ]:
start_time = time.time()
cur.execute(consulta)
resultados_con_indice = cur.fetchall()
tiempo_con_indice = time.time() - start_time


print(f'Tiempo de ejecución con índice: {tiempo_con_indice:.6f} segundos')
print(f'Número de resultados con índice: {len(resultados_con_indice)}')
